In [1]:
import math
import re
import os
import sys

# This is my implementation of standard NB

I applied the theoretical principles of the Naive-Bayes algorithm here in a step by step manner using only pandas, numpy and python libraries along the way. We used sci-kit learn libraries to obtain precision and recall. I obtained very strong results with 83.5% classification accuracy here.

In [2]:
import pandas as pd
train_pos = pd.read_csv('txt_sentoken/pos/train-pos/pos-files-labels.csv', index_col = 'Unnamed: 0')

In [91]:
train_pos.shape

(900, 2)

In [4]:
train_neg = pd.read_csv('txt_sentoken/neg/train-neg/neg-files-labels.csv', index_col = 'Unnamed: 0')

In [92]:
train_neg.shape

(900, 2)

In [6]:
join = pd.concat([train_neg, train_pos])

In [7]:
import numpy as np
input_txt = np.array(join['File Input'])
unique_words_count_pos = {}
unique_words_count_neg = {}
unique_count_pos = 0
unique_count_neg = 0
wrds = []
for i in range(len(input_txt)):
    words = input_txt[i].split()
    # split each input into list easier for iterating through
    wrds.append(words)    

In [9]:
# this use of split has worked
len(wrds[0])

699

In [10]:
# check length of first file with unique words only
len(set(wrds[0]))

340

In [11]:
# get length of total positive and negative docs
# iterate through each file input and if positive add length of doc, vice versa for negative
len_pos = 0
len_neg = 0
labels1 = np.array(join['Label'])
for i in range(len(wrds)):
    if labels1[i] == 1:
        len_pos += len(wrds[i])
    else:
        len_neg += len(wrds[i])

In [12]:
# get unique length of total positive and negative docs
# iterate through each file input and if positive add length of doc, vice versa for negative
len_pos_set = 0
len_neg_set = 0
labels1 = np.array(join['Label'])
for i in range(len(wrds)):
    if labels1[i] == 1:
        #print(len(set(wrds[i])))
        len_pos_set += len(set(wrds[i]))
    else:
        #print(len(set(wrds[i])))
        len_neg_set += len(set(wrds[i]))

In [13]:
len(set(wrds[0]))

340

In [14]:
# get unique words in all docs
uniques = []
for i in range(len(wrds)):
    for j in range(len(wrds[i])):
        if wrds[i][j] not in uniques:
            uniques.append(wrds[i][j])
        else:
            continue

In [15]:
len(uniques)

45241

In [16]:
# now let's split our unique words into the class they lie in (positive or negative)
uniques_pos = []
uniques_neg = []
labels = np.array(join['Label'])
for i in range(len(wrds)):
    if labels[i] == 1:
        for j in range(len(wrds[i])):
            if wrds[i][j] not in uniques_pos:
                uniques_pos.append(wrds[i][j])
            else:
                continue
    else:
        for j in range(len(wrds[i])):
            if wrds[i][j] not in uniques_neg:
                uniques_neg.append(wrds[i][j])
            else:
                continue

In [17]:
# number of unique words in negative docs
len(uniques_neg)

31106

In [18]:
# number of unique words in positive docs
len(uniques_pos)

33172

### Half docs positive, half negative on test set and training set
### So both prior probabilities will be 0.5 

In [19]:
train_pos

,File Input,Label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,youve got mail works alot better than it deser...,1
3,jaws is a rare film that grabs your attention ...,1
4,moviemaking is a lot like being the general ma...,1
...,...,...
895,the soldiers of three kings have taken their c...,1
896,bill condons gods and monsters is a fascinatin...,1
897,i must admit that i was a tad skeptical of goo...,1
898,a cinematic version of one of john irvings nov...,1


In [20]:
join1 = join.copy()
join1

,File Input,Label
0,plot two teen couples go to a church party dri...,0
1,the happy bastards quick movie review damn tha...,0
2,it is movies like these that make a jaded movi...,0
3,quest for camelot is warner bros first feature...,0
4,synopsis a mentally unstable man undergoing ps...,0
...,...,...
895,the soldiers of three kings have taken their c...,1
896,bill condons gods and monsters is a fascinatin...,1
897,i must admit that i was a tad skeptical of goo...,1
898,a cinematic version of one of john irvings nov...,1


In [21]:
# create set to get unique words in each file
# as per notes "For tasks like sentiment, word occurrence is more important than word frequency"
# this is in case we want to try binary NB
input_txt1 = np.array(join1['File Input'])
wrds1 = []
for i in range(len(input_txt)):
    words1 = input_txt[i].split()
    words2 = list(set(words1))
    wrds1.append(words2)

In [22]:
# create dictionaries to get number times a word occurs
# remember standard NB takes frequency of words and not occurrence
unique_wrd_doc_counts_pos = {}
unique_wrd_doc_counts_neg = {}
labels1 = np.array(join1['Label'])
for i in range(len(wrds)):
    if labels1[i] == 1:
        for j in range(len(wrds[i])):
            if wrds[i][j] not in unique_wrd_doc_counts_pos:
                unique_wrd_doc_counts_pos[wrds[i][j]] = 1
            else:
                unique_wrd_doc_counts_pos[wrds[i][j]] += 1
    else:
        for j in range(len(wrds[i])):
            if wrds[i][j] not in unique_wrd_doc_counts_neg:
                unique_wrd_doc_counts_neg[wrds[i][j]] = 1
            else:
                unique_wrd_doc_counts_neg[wrds[i][j]] += 1

In [25]:
len_pos

613316

In [26]:
len_neg

547676

In [27]:
testdf = pd.DataFrame.from_dict(unique_wrd_doc_counts_pos, orient='index')

In [28]:
testdf = testdf.reset_index()
testdf.rename(columns={'index': 'Word', 0: 'FrequencyPositive'}, inplace = True)

In [29]:
testdf

,Word,FrequencyPositive
0,films,1045
1,adapted,25
2,from,2418
3,comic,200
4,books,50
...,...,...
33167,noblewoman,1
33168,fennyman,1
33169,forementioned,1
33170,truelove,1


In [30]:
testdf1 = pd.DataFrame.from_dict(unique_wrd_doc_counts_neg, orient='index')
testdf1 = testdf1.reset_index()
testdf1.rename(columns={'index': 'Word', 0: 'FrequencyNegative'}, inplace = True)
testdf1

,Word,FrequencyNegative
0,plot,779
1,two,787
2,teen,68
3,couples,15
4,go,468
...,...,...
31101,belch,1
31102,secondclass,1
31103,maids,1
31104,creeping,1


In [31]:
merged_df = pd.merge(testdf, testdf1, on="Word", how='outer')
merged_df

,Word,FrequencyPositive,FrequencyNegative
0,films,1045.0,828.0
1,adapted,25.0,14.0
2,from,2418.0,2032.0
3,comic,200.0,141.0
4,books,50.0,23.0
...,...,...,...
45236,exspecial,NaN,1.0
45237,yucked,NaN,1.0
45238,belch,NaN,1.0
45239,secondclass,NaN,1.0


In [32]:
merged_df = merged_df.fillna(0)

In [33]:
merged_df.dtypes

Word                  object
FrequencyPositive    float64
FrequencyNegative    float64
dtype: object

In [34]:
merged_df['FrequencyPositive'] = merged_df['FrequencyPositive'].astype('int64')
merged_df['FrequencyNegative'] = merged_df['FrequencyNegative'].astype('int64')

In [35]:
merged_df

,Word,FrequencyPositive,FrequencyNegative
0,films,1045,828
1,adapted,25,14
2,from,2418,2032
3,comic,200,141
4,books,50,23
...,...,...,...
45236,exspecial,0,1
45237,yucked,0,1
45238,belch,0,1
45239,secondclass,0,1


In [36]:
# calculate word probabilities
pos_prob = []
neg_prob = []
vocab_pos = len(uniques_pos)
vocab_neg = len(uniques_neg)
vocab = len(uniques)
words_prob = np.array(merged_df['Word'])
pos_counts = np.array(merged_df['FrequencyPositive'])
neg_counts = np.array(merged_df['FrequencyNegative'])
for i in range(len(words_prob)):
    # laplace smoothing to handle zero probabilities
    # len_pos is 613316 from earlier
    pos_probs = (pos_counts[i] + 1) / (len_pos + vocab)
    # calculate probabilities both negative and positive for each word
    # len_neg is 547676
    neg_probs = (neg_counts[i] + 1) / (len_neg + vocab)
    pos_prob.append(pos_probs)
    neg_prob.append(neg_probs)

In [37]:
len(pos_prob)

45241

In [38]:
merged_df['ProbabilityPositive'] = pos_prob
merged_df['ProbabilityNegative'] = neg_prob

In [39]:
merged_df

,Word,FrequencyPositive,FrequencyNegative,ProbabilityPositive,ProbabilityNegative
0,films,1045,828,0.001588,0.001398
1,adapted,25,14,0.000039,0.000025
2,from,2418,2032,0.003673,0.003429
3,comic,200,141,0.000305,0.000239
4,books,50,23,0.000077,0.000040
...,...,...,...,...,...
45236,exspecial,0,1,0.000002,0.000003
45237,yucked,0,1,0.000002,0.000003
45238,belch,0,1,0.000002,0.000003
45239,secondclass,0,1,0.000002,0.000003


### We need to ignore unknown words in test data

Let's read in positive and negative test files first.

In [40]:
## Read in test files
test_pos = pd.read_csv('txt_sentoken/pos/test-pos/pos-test-files-labels.csv', index_col = 'Unnamed: 0')

In [93]:
test_pos.head()

,File Input,Label
0,in 1912 a ship set sail on her maiden voyage a...,1
1,the start of this movie reminded me of parts f...,1
2,note some may consider portions of the followi...,1
3,robert altmans cookies fortune is that rare mo...,1
4,well ill be damned the canadians can make a go...,1


In [42]:
test_neg = pd.read_csv('txt_sentoken/neg/test-neg/neg-test-files-labels.csv', index_col = 'Unnamed: 0')

In [94]:
test_neg.head()

,File Input,Label
0,there may not be a critic alive who harbors as...,0
1,renee zellweger stars as sonia a young jewish ...,0
2,therere so many things to criticize about i do...,0
3,dont let this movie fool you into believing th...,0
4,its a good thing most animated scifi movies co...,0


In [44]:
# combine whole test set of documents
join_test = pd.concat([test_neg, test_pos])
join_test

,File Input,Label
0,there may not be a critic alive who harbors as...,0
1,renee zellweger stars as sonia a young jewish ...,0
2,therere so many things to criticize about i do...,0
3,dont let this movie fool you into believing th...,0
4,its a good thing most animated scifi movies co...,0
...,...,...
95,wow what a movie its everything a movie can be...,1
96,richard gere can be a commanding actor but hes...,1
97,glorystarring matthew broderick denzel washing...,1
98,steven spielbergs second epic film on world wa...,1


### Next step - removing words not in training vocab

In [45]:
input_txt_test = np.array(join_test['File Input'])
unknown_rm = []
for i in range(len(input_txt_test)):
    lst = []
    wrds = input_txt_test[i].split()
    for j in range(len(wrds)):
        if wrds[j] in uniques:
            lst.append(wrds[j])
        else:
            continue
    str2 = ' '.join([str(elem) for elem in lst])
    unknown_rm.append(str2)

In [46]:
input_txt_test[0] == unknown_rm[0]

False

In [47]:
len(input_txt_test[0].split())

481

In [48]:
len(unknown_rm[0].split())

462

In [49]:
# update test file inputs with unknown words removed
join_test['File Input'] = unknown_rm

In [50]:
merged_df

,Word,FrequencyPositive,FrequencyNegative,ProbabilityPositive,ProbabilityNegative
0,films,1045,828,0.001588,0.001398
1,adapted,25,14,0.000039,0.000025
2,from,2418,2032,0.003673,0.003429
3,comic,200,141,0.000305,0.000239
4,books,50,23,0.000077,0.000040
...,...,...,...,...,...
45236,exspecial,0,1,0.000002,0.000003
45237,yucked,0,1,0.000002,0.000003
45238,belch,0,1,0.000002,0.000003
45239,secondclass,0,1,0.000002,0.000003


In [56]:
np.log(merged_df['ProbabilityPositive'].max())

-2.8780526680236895

In [57]:
merged_df['ProbabilityPositive'].max()

0.05624418235627288

In [58]:
# using log probabilities for classification
merged_df['ProbabilityPositiveLog'] = np.log(merged_df['ProbabilityPositive'])
merged_df['ProbabilityNegativeLog'] = np.log(merged_df['ProbabilityNegative'])

In [59]:
merged_df

,Word,FrequencyPositive,FrequencyNegative,ProbabilityPositive,ProbabilityNegative,ProbabilityPositiveLog,ProbabilityNegativeLog
0,films,1045,828,0.001588,0.001398,-6.445078,-6.572590
1,adapted,25,14,0.000039,0.000025,-10.139710,-10.584760
2,from,2418,2032,0.003673,0.003429,-5.606697,-5.675542
3,comic,200,141,0.000305,0.000239,-8.094501,-8.336983
4,books,50,23,0.000077,0.000040,-9.465981,-10.114756
...,...,...,...,...,...,...,...
45236,exspecial,0,1,0.000002,0.000003,-13.397806,-12.599663
45237,yucked,0,1,0.000002,0.000003,-13.397806,-12.599663
45238,belch,0,1,0.000002,0.000003,-13.397806,-12.599663
45239,secondclass,0,1,0.000002,0.000003,-13.397806,-12.599663


### Calculating document probabilities and polarity labels

In [60]:
# typically takes a while to run this cell
input_txt_test = np.array(join_test['File Input'])
probabilities_pos = []
probabilities_neg = []
predictions = []
for i in range(len(input_txt_test)):
    # prior probability for both is 0.5 as half training docs are pos and half negative
    prior_pos = 0.5
    prior_neg = 0.5
    # split on each word in doc
    wrds = input_txt_test[i].split()
    for j in range(len(wrds)):
        # find positive probability of word from our dataframe of positive and negative probabilities
        p = merged_df.loc[merged_df['Word'] == wrds[j]].ProbabilityPositiveLog.item()
        # likewise, find negative probability of word from our dataframe of positive and negative probabilities
        n = merged_df.loc[merged_df['Word'] == wrds[j]].ProbabilityNegativeLog.item()
        # add log probability found by each prior probability (rules of log)
        # log(a)+log(b)=log(ab)
        prior_pos += p
        prior_neg += n
    # added these lists in just to double check end results!
    probabilities_pos.append(prior_pos)
    probabilities_neg.append(prior_neg)
    # if prior probability in end is greater than negative assign label 1 (positive)
    if prior_pos > prior_neg:
        predictions.append(1)
    elif prior_pos < prior_neg:
        predictions.append(0)
    # handle equal values if they occur (very unlikely but just in case)
    else:
        predictions.append(-1)

In [61]:
len(predictions)

200

In [62]:
#probabilities_neg

In [63]:
join_test['Predictions'] = predictions
join_test['ProbPosNormal'] = probabilities_pos
join_test['ProbNegNormal'] = probabilities_neg

## Evaluation

In [66]:
pred = np.array(join_test['Predictions'])
actual = np.array(join_test['Label'])
count = 0
for i in range(len(pred)):
    # check if prediction is equal to actual label and add count + 1 if this is true
    if pred[i] == actual[i]:
        count += 1
    else:
        continue

In [67]:
accc = (count / len(pred))
print('Accuracy of Naive Bayes Classifier with no stop word removal is ' + str(accc))

Accuracy of Naive Bayes Classifier with no stop word removal is 0.835


In [87]:
from sklearn.metrics import precision_score
precision = precision_score(actual, pred)
print('Precision of Naive-Bayes without stop word removal is: %.3f' % precision)

Precision of Naive-Bayes without stop word removal is: 0.845


In [88]:
from sklearn.metrics import recall_score
recall = recall_score(actual, pred)
print('Recall of Naive-Bayes without stop word removal is: %.3f' % recall)

Recall of Naive-Bayes without stop word removal is: 0.820


In [95]:
merged_df.head()

,Word,FrequencyPositive,FrequencyNegative,ProbabilityPositive,ProbabilityNegative,ProbabilityPositiveLog,ProbabilityNegativeLog,TotalCounts
22,the,37039,31356,0.056244,0.052886,-2.878053,-2.939617,68395
32,a,18134,16031,0.027537,0.027039,-3.592207,-3.610468,34165
45,and,17695,13953,0.026871,0.023534,-3.616712,-3.749288,31648
8,of,16623,13797,0.025243,0.023271,-3.679204,-3.760531,30420
51,to,14729,13778,0.022367,0.023239,-3.800165,-3.761909,28507


### Let's try remove stop words and see does this improve our NB classifier

We will take the top 50 most frequent words and remove these.

In [69]:
# get total counts of words from both positive and negative documents to help pinpoint stop words
freq_p = np.array(merged_df['FrequencyPositive'])
freq_n = np.array(merged_df['FrequencyNegative'])
count_all = []
for i in range(len(freq_p)):
    num = freq_p[i] + freq_n[i]
    count_all.append(num)

In [70]:
#count_all

In [71]:
merged_df['TotalCounts'] = count_all

In [72]:
merged_df.head()

,Word,FrequencyPositive,FrequencyNegative,ProbabilityPositive,ProbabilityNegative,ProbabilityPositiveLog,ProbabilityNegativeLog,TotalCounts
0,films,1045,828,0.001588,0.001398,-6.445078,-6.572590,1873
1,adapted,25,14,0.000039,0.000025,-10.139710,-10.584760,39
2,from,2418,2032,0.003673,0.003429,-5.606697,-5.675542,4450
3,comic,200,141,0.000305,0.000239,-8.094501,-8.336983,341
4,books,50,23,0.000077,0.000040,-9.465981,-10.114756,73


In [73]:
merged_df = merged_df.sort_values(by=['TotalCounts'], ascending=False)

In [74]:
stop_words = merged_df.loc[merged_df['TotalCounts'] >= 1000][:50]
#stop_words

In [75]:
stop_words_rm = list(stop_words['Word'])
test_input_stop = np.array(join_test['File Input'])
rm_stp_words = []
for i in range(len(test_input_stop)):
    wrds = test_input_stop[i].split()
    filtered = []
    for j in range(len(wrds)):
        if wrds[j] not in stop_words_rm:
            filtered.append(wrds[j])
        else:
            continue
    str2 = ' '.join([str(elem) for elem in filtered])
    rm_stp_words.append(str2)

In [76]:
join_test['File Input'] = rm_stp_words

In [77]:
# typically takes a while to run this cell
# let's now get our classifications for our documents with stop words removed
input_txt_test = np.array(join_test['File Input'])
probabilities_pos = []
probabilities_neg = []
predictions = []
for i in range(len(input_txt_test)):
    # prior probability for both is 0.5 as half training docs are pos and half negative
    prior_pos = 0.5
    prior_neg = 0.5
    # split on each word in doc
    wrds = input_txt_test[i].split()
    for j in range(len(wrds)):
        # find positive probability of word from our dataframe of positive and negative probabilities
        p = merged_df.loc[merged_df['Word'] == wrds[j]].ProbabilityPositiveLog.item()
        # likewise, find negative probability of word from our dataframe of positive and negative probabilities
        n = merged_df.loc[merged_df['Word'] == wrds[j]].ProbabilityNegativeLog.item()
        # add log probability found by each prior probability (rules of log)
        # log(a)+log(b)=log(ab)
        prior_pos += p
        prior_neg += n
    # added these lists in just to double check end results!
    probabilities_pos.append(prior_pos)
    probabilities_neg.append(prior_neg)
    # if prior probability in end is greater than negative assign label 1 (positive)
    if prior_pos > prior_neg:
        predictions.append(1)
    elif prior_pos < prior_neg:
        predictions.append(0)
    # handle equal values if they occur
    else:
        predictions.append(-1)

In [78]:
join_test['PredictionsStopRemoved'] = predictions

In [79]:
pred_stop = np.array(join_test['PredictionsStopRemoved'])
actual = np.array(join_test['Label'])
count_stop = 0
for i in range(len(pred_stop)):
    if pred_stop[i] == actual[i]:
        count_stop += 1
    else:
        continue

In [80]:
accc_stop = (count_stop / len(pred_stop))
print('Accuracy of Naive Bayes Classifier with stop words removed is ' + str(accc_stop))

Accuracy of Naive Bayes Classifier with stop words removed is 0.825


In [89]:
precision1 = precision_score(actual, pred_stop)
print('Precision of Naive-Bayes with stop word removal is: %.3f' % precision1)

Precision of Naive-Bayes with stop word removal is: 0.857


In [90]:
recall1 = recall_score(actual, pred_stop)
print('Recall of Naive-Bayes with stop word removal is: %.3f' % recall1)

Recall of Naive-Bayes with stop word removal is: 0.780


In [81]:
#join_test.to_csv('analysis-data.csv')

In [82]:
#merged_df.to_csv('word-probs.csv')